%load_ext tutormagic

# Non-Local Assignment

## The Effect of Nonlocal Statements

Future assignments to names that are declared `nonlocal` change its pre-existing binding in the **first non-local frame** of the current environment in which that name is bound. 

In Python Docs, this **first non-loca frame** is called an "enclosing scope".

It is possible to have multiple names separated by comma `,` accompanying the `nonlocal` statement.

In [ ]:
nonlocal <name1>, <name2>, <name3>, etc.

And there are restrictions to the names that we can declare `nonlocal`.

From Python 3 language reference: "Names listed in a nonlocal statement must...

1. ...refer to pre-existing bindings in an enclosing scope."
    * In other words, the name must have been previously used.
2. ...not collide with pre-existing bindings in the local scope (current frame)
    * We can't declare a name to be nonlocal if it already exists in the current frame. The name must be in the parent frame, or any frame between the global frame to the current frame.

## The Many Meanings of Assignment Statements

Let's say we see the following assignment statement,

In [ ]:
x = 2

Based on the context of the program, this assignment statement might do different things. Here are a list of many meanings of assignment statements.

| Status | Effect |
| ---- | ---- |
| - No nonlocal statement <br> -`x` **is not** bound locally | Create a new binding from name `x` to object 2 in the first frame of the current environment |
| - No nonlocal statement <br> -`x` **is** bound locally | Re-bind name `x` to object `2` in the first frame of the current environment |
| - Nonlocal `x` <br> -`x` **is** bound in a non-local frame | Re-bind `x` to `2` in the first non-local frame of the current environment in which `x` is bound |
| - Nonlocal `x` <br> -`x` **is not** bound in a non-local frame | `SyntaxError`: no binding for nonlocal `x` found |
| - Nonlocal `x` <br> -`x` **is** bound in a non-local frame <br> -`x` also bound locally | `SyntaxError`: name `x` is parameter and nonlocal |

The error messages might be different depending on the implementation of Python. Regardless, they correspond to the 2 conditions:

1. `x` must have already been bound
2. but it can't be bound in the current frame

## Python Particulars

The general rules about binding nonlocally in previous section applies to many different languages. However, in Python, there're specific features that we need to be wary of.

Python pre-computes which frame contains each name before executing the body of a function. 

Within the body of a function, all instances of a name must refer to the same frame (can't have the same name in the same function body refer to 2 different frames). 

<img src = 'if_local.jpg' width = 800/>

If we try to write `make_withdraw` without the `nonlocal` statement, the error would occur after Python is done checking if `amount` > `balance`. Here, Python thinks `balance` is a local name, thus it only looks at the `withdraw` frame. Thus, we'll obtain the `UnboundLocalError` message.

If we remove the `balance = balance - amount` line,

In [ ]:
def make_withdraw(balance):
    def withdraw(amount):
        # nonlocal balance
        if amount > balance:
            return 'Insufficient funds'
        # balance = balance - amount
        return balance
    return withdraw

withdraw = make_withdraw(100)
withdraw(25)
withdraw(25)
withdraw(60)
withdraw(15)


Then we won't run into error since there's no assignment involved.

## Mutable Values & Persistent Local State

Using a `nonlocal` statement is not the only way to create a mutable function. Mutable values can be changed without a nonlocal statement.

Here, we use an existing mutable value,

In [ ]:
def make_withdraw_list(balance):
    b = [balance] #puts balance in a list 
    def withdraw(amount):
        if amount > b[0]:
            return 'Insufficient funds'
        b[0] = b[0] - amount
        return b[0]
    return withdraw

withdraw = make_withdraw_list(100)
withdraw(25)

Above, we put `balance` in a list of only one element.

<img src = 'list.jpg' width = 500/>

This `b` can change because it is a list. We can change it using element assignment.

Here we don't need `nonlocal` assignment because we don't change any of the following within the body of `withdraw`:
1. What `b` is bound to
2. What `balance` is bound to 
3. What `withdraw` is bound to